Importing the dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
#loading the dataset into pandas dataframe
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [3]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
# dataset information
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [6]:
# checking no.of missing values
credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


No missing values is observed

In [7]:
#distribution of legit and fraud transaction
credit_card_data['Class'].value_counts()

,count
Class,
0,284315
1,492


- 0 = normal transactions
- 1 = fraud transactions
* the fraud transactions is less, this is an unbalanced data

In [9]:
# separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [10]:
print(legit.shape, fraud.shape)

(284315, 31) (492, 31)


In [11]:
# getting statistical measures for the data
legit.Amount.describe()

,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [12]:
fraud.Amount.describe()

,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [13]:
#compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Dealing with unbalanced data - undersampling

Building a sample dataset that will have similar distribution of normal and fraud transaction

In [14]:
legit_sample = legit.sample(n=492)

Concat two dataframes

In [15]:
new_dataset = pd.concat([legit_sample, fraud], axis=0) #axis = 0 -> values will be added row wise, if axis = 1 -> then column

In [16]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
163555,116039.0,1.886940,-1.406811,-0.088318,-0.786804,-0.863695,1.656361,-1.800250,0.661292,0.154385,0.892483,0.701839,0.338634,0.048846,-0.073759,1.517829,-0.925705,-0.278489,1.213801,-1.951437,-0.589445,-0.015853,0.486189,0.316563,-1.648432,-0.914755,0.711361,0.055389,-0.051415,45.90,0
224889,144010.0,-1.248820,-0.364148,-0.648318,1.476239,-4.222154,3.122057,5.322583,-0.851598,-0.863321,-0.917907,-0.947975,-0.617993,0.293411,-0.044783,1.005372,-1.073383,0.713648,0.199673,2.856504,0.539685,0.030709,0.515957,0.381524,0.792117,0.766373,1.006565,0.366411,-0.191433,1143.55,0
110323,71772.0,-4.224018,2.855178,-2.907353,-1.644928,0.437225,3.088031,-2.104243,2.404936,-0.564214,-0.452184,-1.096501,0.737084,0.261945,1.754533,0.444713,1.346008,-0.173428,0.324850,-0.409130,-0.684041,0.939931,-0.662003,0.244468,0.995836,0.070149,0.111231,-1.721170,-0.620121,9.99,0
35148,37988.0,1.027157,-0.135272,0.331489,1.334000,-0.305197,0.118615,-0.099735,0.227719,0.313800,0.077504,0.835833,0.207366,-2.109399,0.651812,-0.579348,-0.430240,0.121966,-0.238522,-0.079714,-0.202399,-0.042850,-0.125835,-0.094582,-0.010918,0.551342,-0.328879,0.014471,0.010176,56.80,0
171876,120854.0,1.998105,0.086954,-1.910761,0.463361,0.371090,-1.187441,0.326530,-0.372692,0.565928,-0.507803,-0.608052,0.115523,0.378749,-0.799298,0.946109,0.252986,0.289324,0.513452,-0.366913,-0.096580,0.203165,0.694119,-0.103549,-0.472148,0.287674,-0.078607,-0.005434,-0.027996,44.38,0


In [18]:
new_dataset['Class'].value_counts()

,count
Class,
0,492
1,492


In [19]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,97797.231707,-0.135701,-0.096485,-0.042591,0.019638,0.074320,-0.006819,0.098784,0.051257,0.150815,-0.048800,-0.067741,0.045280,0.029019,-0.085074,-0.067865,0.022811,0.002209,0.059096,0.021645,0.037923,0.011433,0.015637,0.034004,0.004888,-0.003442,-0.003667,0.001978,0.001645,110.630488
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Splitting the data into features and target

In [20]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [21]:
print(X)

            Time        V1        V2  ...       V27       V28   Amount
163555  116039.0  1.886940 -1.406811  ...  0.055389 -0.051415    45.90
224889  144010.0 -1.248820 -0.364148  ...  0.366411 -0.191433  1143.55
110323   71772.0 -4.224018  2.855178  ... -1.721170 -0.620121     9.99
35148    37988.0  1.027157 -0.135272  ...  0.014471  0.010176    56.80
171876  120854.0  1.998105  0.086954  ... -0.005434 -0.027996    44.38
...          ...       ...       ...  ...       ...       ...      ...
279863  169142.0 -1.927883  1.125653  ...  0.292680  0.147968   390.00
280143  169347.0  1.378559  1.289381  ...  0.389152  0.186637     0.76
280149  169351.0 -0.676143  1.126366  ...  0.385107  0.194361    77.89
281144  169966.0 -3.113832  0.585864  ...  0.884876 -0.253700   245.00
281674  170348.0  1.991976  0.158476  ...  0.002988 -0.015309    42.53

[984 rows x 30 columns]


In [22]:
print(Y)

163555    0
224889    0
110323    0
35148     0
171876    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


Train Test split

In [23]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [24]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


Model Training - Logistic

In [25]:
model = LogisticRegression()

In [26]:
#training the logistic regression model wiht training data
model.fit(X_train,Y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Model Evaluation - based on accuracy score



In [27]:
#accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [28]:
print(training_data_accuracy)

0.9415501905972046


In [29]:
#accuracy on testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [30]:
print(testing_data_accuracy)

0.9289340101522843
